# 각 데이터별 Docs 5개 생성하기
- 정답 context(1) + negative samples(4)

In [ ]:
import random
import ast
import pandas as pd
from openai import OpenAI
from google.colab import userdata
import os
import re
from tqdm import tqdm

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
# klue_mrc_context_question_negative_samples.csv
!gdown 1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc

Downloading...
From (original): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc
From (redirected): https://drive.google.com/uc?id=1Jy5FX41lSF9VH-LUeCPTgDAn9flXQqHc&confirm=t&uuid=27b7d8d4-3781-4d5a-97b1-14ed7d114845
To: /content/klue_mrc_context_question_negative_samples.csv
100% 161M/161M [00:04<00:00, 39.1MB/s]


In [ ]:
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv('klue_mrc_context_question_negative_samples.csv')

# 앞에서 300개만 추출
df_300 = df.iloc[:300]
df_300

,context,question,negative_samples
0,올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도...,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,['궤도물리학은 계절의 지속 기간이 지구의 궤도가 지점과 분점 사이의 공간을 휩쓸고...
1,부산시와 (재)부산정보산업진흥원(원장 이인숙)이 ‘2020~2021년 지역SW서비스...,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,"['한국인터넷진흥원(KISA, 원장 김석환)은 과학기술정보통신부(장관 최기영)와 포..."
2,"미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에...",로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,['1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 ...
3,시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인으...,개막전에서 3안타 2실점을 기록해서 패한 선수는?,['1960년에 주니치에 입단하여 같은 해 1960년 5월 7일 다이요 웨일스전에서...
4,유명 맛집 이름을 달고 나온 편의점 자체상표(PB) 라면이 인기를 끌고 있다. ‘검...,컵라면 매출에서 불닭볶음면을 이긴 상품은?,['‘짜파구리’로 시작된 ‘나만의 레시피’ 열풍이 올해 식품시장을 주도한 것으로 나...
...,...,...,...
295,애류헌(愛流軒·Erewhon)이란 이름엔 두 가지 의미가 있다. 한자로는 ‘사랑이 ...,애류헌에서 판매되는 전통주는 몇 종인가?,['애경그룹이 9일 창립 60주년을 맞았다.애경은 1954년 비누 제조사 애경유지공...
296,막말 이후 부국강병 명분으로 해마다 증세가 이뤄지는 가운데 1881년 재무경에 취임...,마쓰카타 마사요시의 재정정책으로 가장 많은 타격을 입은 산업분야는?,['일본의 3분기 경제성장률이 지난달 발표된 잠정치를 밑돈 것으로 나타났다. 아베 ...
297,다중 계정 사용자의 활동을 막기 위해 무작정 계정을 두 개 이상 만드는 행위를 차단...,다중 계정 사용자의 판별을 불가능하게 하는 것은?,['해리성 정체성 장애\n명확히 구별할 수 있는 복수(複数)의 인격이 동일인에게 존...
298,"지난 10월, 국내외 많은 화제 속에 크랭크인 하여 12월 막바지 촬영 중인 판타지...",가시리잇고가 상영을 시작한 달은?,['에이앤이 네트웍스의 첫 한국 제작 투자 드라마 <편의점 샛별이>가 전 세계적으로...


In [ ]:
for idx, row in df_300.iterrows():
    col = row['negative_samples']
    print(type(col))
    col = ast.literal_eval(row['negative_samples'])
    print(type(col))
    break

<class 'str'>
<class 'list'>


In [ ]:
# 각 row에 대해 docs 리스트를 만들고 DataFrame에 저장
def make_docs(row):
    negatives = row['negative_samples'] # str 타입
    # negative_samples가 str이면 리스트로 변환
    if isinstance(negatives, str):
        negatives = ast.literal_eval(negatives)
    docs = [row['context']] + negatives # 정답 + 오답(나름 정답과 유사한)
    random.shuffle(docs)
    return docs

df_300['docs'] = df_300.apply(make_docs, axis=1)
df_300[['question', 'docs']]

/tmp/ipython-input-29-1366817783.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_300['docs'] = df_300.apply(make_docs, axis=1)


,question,docs
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,[“물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극...
1,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,[청강문화산업대는 지난해 5월 컴퓨터그래픽(CG) 소프트웨어(SW)업체 에프엑스기어...
2,로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,[사이코-사이버네틱스(Psycho-Cybernetics)는 정신적인 자동유도장치라는...
3,개막전에서 3안타 2실점을 기록해서 패한 선수는?,[시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인...
4,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[영화관 안에서만 주로 접했던 팝콘이 편의점 진열대의 윗자리를 차지하는 대표 스낵으...
...,...,...
295,애류헌에서 판매되는 전통주는 몇 종인가?,[국립종자원은 이달 초 제주도에 지원을 개설했다. 아열대 작물을 개량해 특허권을 확...
296,마쓰카타 마사요시의 재정정책으로 가장 많은 타격을 입은 산업분야는?,[막말 이후 부국강병 명분으로 해마다 증세가 이뤄지는 가운데 1881년 재무경에 취...
297,다중 계정 사용자의 판별을 불가능하게 하는 것은?,[개인정보 불법 거래가 여전한 데다 금융사기가 증가세를 보이면서 ‘금융사기의 필수품...
298,가시리잇고가 상영을 시작한 달은?,"[‘일본의, 일본에 의한, 일본을 위한 애니메이션.’일본 미야자키 하야오 감독(72..."


In [ ]:
df_300.loc[0, 'docs'][1]

'2017년까지 강원 양양과 충남 태안, 경남 남해 등 12개 해안 지역거점 12곳이 휴양·체험·생태벨트로 조성된다.국토교통부는 지난해부터 동서남해안 일대에 조성 중인 ‘휴양·체험·생태벨트’에 4곳을 추가해 총 12곳을 개발한다고 25일 발표했다.국토부는 지난해 동해 망상과 영덕 고래불, 강릉 심곡, 울산 진하, 전북 고창, 전남 진도항, 전남 고흥, 경남 거제 등 8곳을 휴양·체험·생태벨트 거점으로 개발하기로 하고 사업을 진행 중이다.새로 추가된 양양에는 오색 자연휴양체험지구가 조성된다. 올해부터 2017년까지 국비와 지방비 등 300억원을 투자해 양양군 서면 오색리 35만㎡ 일대에 캠핑장 등 자연휴양 체험시설을 짓는다. 태안군 소원면 만리포해수욕장 내 31만1853㎡에는 180억원을 들여 해안도로와 탐방로를 건설한다.또 남해군 미조면 미조리 조도와 호도 일원 24만㎡는 316억원을 투입해 접안시설과 다이어트 센터, 수상가옥 등을 조성해 남해 다이어트 보물섬으로 거듭나게 된다. 부산 해운대 동해남부선 폐선 부지 26만8555㎡에 240억원을 들여 해안 절경을 즐길 수 있는 공원 시설을 조성한다.국토부는 이번에 새로 추가된 4곳을 포함해 12곳에 2017년까지 국비와 지방지 2086억원 투입할 계획이다. 김영우 국토부 해안권발전지원과장은 “휴양·체험·생태벨트가 조성되면 국민의 여가생활 공간이 확대되고 지역 경제도 활성화돼 균형 발전의 기반이 될 것”이라며 “앞으로도 해안권별로 사업을 발굴해 개발할 계획”이라고 설명했다.'

In [ ]:
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

user_prompts = []
system_prompts = []

for idx, row in df_300.iterrows():
    docs = row['docs']  # 이미 섞인 리스트
    # 문서에 번호 붙이기
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)
    # user_prompt 생성
    user_prompt = f"""질문: {row['question']}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""
    # 리스트에 추가
    user_prompts.append(user_prompt)
    system_prompts.append(system_prompt)

# 예시 출력
print("system_prompt:")
print(system_prompts[0])
print("\nuser_prompt:")
print(user_prompts[0])

system_prompt:
당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.

user_prompt:
질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

docs:
doc1: “물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극항로 운항에 나선 스테나폴라리스호(號)가 베링해협을 지나 동해에 들어선 19일. 젊은 시절 10년간 항해 경험이 있는 남청도 해양대 교수는 유난히 푸른 바다를 가리키며 이렇게 말했다. 이제 목적지인 광양항까지는 이틀 남았다. 스테나폴라리스는 광양항에 도착한다. 지난 9월16일 밤 러시아 우스트루가를 떠난 지 35일 만이다. 바스코 알렉산더 선장은 “올해는 작년보다 해빙(海氷) 양이 늘어난 데다 쇄빙선 일정에 차질이 생겨 예상보다 약간 늦어졌다”며 “그래도 수에즈 운하를 통과하는 (북유럽~한국) 노선에 비해 8일 정도 운항일수를 줄였다”고 했다. 남 교수는 이번 북극항로 시범운항의 성과에 대해 “전체적으로 수에즈 운하를 지나는 항로에 비해 20만달러 정도 적게 들어간 것으로 파악하고 있다. 이 정도면 경제성이 충분하다”는 견해를 피력했다. 이승헌 현대글로비스 해기사는 “북극항로는 1만5500㎞ 정도로 수에즈 운하를 지나는 항로보다 7000㎞나 짧아 연료

In [ ]:
df_300['user_prompt'] = user_prompts
df_300['system_prompt'] = system_prompts
df_300[['question', 'docs', 'system_prompt', 'user_prompt']].head()

/tmp/ipython-input-32-1152744027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_300['user_prompt'] = user_prompts
/tmp/ipython-input-32-1152744027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_300['system_prompt'] = system_prompts


,question,docs,system_prompt,user_prompt
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,[“물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?\n\ndoc...
1,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,[청강문화산업대는 지난해 5월 컴퓨터그래픽(CG) 소프트웨어(SW)업체 에프엑스기어...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?\n\ndocs...
2,로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,[사이코-사이버네틱스(Psycho-Cybernetics)는 정신적인 자동유도장치라는...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?\...
3,개막전에서 3안타 2실점을 기록해서 패한 선수는?,[시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 개막전에서 3안타 2실점을 기록해서 패한 선수는?\n\ndocs:\ndoc1...
4,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[영화관 안에서만 주로 접했던 팝콘이 편의점 진열대의 윗자리를 차지하는 대표 스낵으...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",질문: 컵라면 매출에서 불닭볶음면을 이긴 상품은?\n\ndocs:\ndoc1: 영화...


In [ ]:
client = OpenAI()

# 응답 저장 리스트
responses = []

for i, user_prompt in enumerate(user_prompts[:3]):
    response = client.chat.completions.create(
        model="gpt-4o", # gpt-4o-mini 사용하는 경우, 적절한 답을 못찾는 경우도 있다.
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    answer = response.choices[0].message.content
    responses.append(answer)
    print(f"\n=== 예제 {i+1} ===")
    print("모델 응답:\n", answer)



=== 예제 1 ===
모델 응답:
 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 약 한 달가량입니다. 이는 장마전선이 형성되어 한반도 중남부를 오르내리며 비를 뿌리는 기간을 의미합니다 [[doc4]].

=== 예제 2 ===
모델 응답:
 성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는 과학기술정보통신부입니다 [[doc4]].

=== 예제 3 ===
모델 응답:
 로버트 헨리 딕이 1946년에 매사추세츠 공과대학교에서 개발한 것은 레이다입니다. 그는 제2차 세계대전 중에 매사추세츠 공과대학교 방사선 연구소에서 레이다를 개발하였고, 딕 복사계와 마이크로파 수신기를 설계하였습니다 [[doc2]].


In [ ]:
def extract_doc_citations(text):
    """
    [[doc1]], [[doc2]], ... 형식의 인용 문서번호만 리스트로 반환
    """
    # 정규표현식: 대소문자 구분 없이 [[doc숫자]]
    citations = re.findall(r'\[\[(doc\d+)\]\]', text, flags=re.IGNORECASE)
    return list(set(citations))  # 중복 제거, 원하면 정렬도 가능

In [ ]:
for i, (user_prompt, answer) in enumerate(zip(user_prompts[:3], responses)):
    cited_docs = extract_doc_citations(answer)
    print(f"\n=== 예제 {i+1} ===")
    print("user_prompt:\n", user_prompt)
    print("\n모델 응답:\n", answer)
    print("인용된 문서 번호:", cited_docs)


=== 예제 1 ===
user_prompt:
 질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

docs:
doc1: “물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극항로 운항에 나선 스테나폴라리스호(號)가 베링해협을 지나 동해에 들어선 19일. 젊은 시절 10년간 항해 경험이 있는 남청도 해양대 교수는 유난히 푸른 바다를 가리키며 이렇게 말했다. 이제 목적지인 광양항까지는 이틀 남았다. 스테나폴라리스는 광양항에 도착한다. 지난 9월16일 밤 러시아 우스트루가를 떠난 지 35일 만이다. 바스코 알렉산더 선장은 “올해는 작년보다 해빙(海氷) 양이 늘어난 데다 쇄빙선 일정에 차질이 생겨 예상보다 약간 늦어졌다”며 “그래도 수에즈 운하를 통과하는 (북유럽~한국) 노선에 비해 8일 정도 운항일수를 줄였다”고 했다. 남 교수는 이번 북극항로 시범운항의 성과에 대해 “전체적으로 수에즈 운하를 지나는 항로에 비해 20만달러 정도 적게 들어간 것으로 파악하고 있다. 이 정도면 경제성이 충분하다”는 견해를 피력했다. 이승헌 현대글로비스 해기사는 “북극항로는 1만5500㎞ 정도로 수에즈 운하를 지나는 항로보다 7000㎞나 짧아 연료비가 30만달러 정도 덜 들어간다”며 “하지만 쇄빙선료와 아이스파일럿비 등을 통행료로 내야 하고 (사고 위험에 대비한) 보험료도 훨씬 높기 때문에 이 부분을 얼마나 줄이느냐가 북극항로의 경제성을 좌우할 것”이라고 말했다. 북극항로 통행료에 대해 패트릭 스반 스테나해운 매니저는 “스테나해운의 북극항로 운항 실적이 많아 이번 통행료는 t당 5달러를 적용해 총 20만달러 정도 들었다”며 “하지만 운항실적이 없는 선사는 훨씬 비싼 비용을 내야 한다”고 했다. 남 교수는 “선사의 운항실적 등 복잡한 규정에 따라 통행료가 달라지는 현재의 관행이 바뀌어야 한다”고 지적했다.북극항로의 안전성에 대해서는 긍정적인 평가가 나왔다. 하지만 쇄빙선 운항시스템은 개선해야 할 점으로 지적됐다. 남 교수는 “쇄빙선이 당

In [ ]:
import time

# 응답 저장 리스트
responses = []

for i, user_prompt in enumerate(tqdm(user_prompts, desc="응답 생성 중", ncols=80)):
    response = client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    answer = response.choices[0].message.content
    responses.append(answer)
    # time.sleep(1)

응답 생성 중: 100%|███████████████████████████| 300/300 [11:29<00:00,  2.30s/it]


In [ ]:
len(responses)

300

In [ ]:
# 1. answer 컬럼 추가
df_300['answer'] = responses

# 2. doc_citations 컬럼 추가
df_300['doc_citations'] = df_300['answer'].apply(extract_doc_citations)

# 3. 결과 일부 확인
df_300[['question', 'answer', 'docs', 'doc_citations']].head()

/tmp/ipython-input-38-2805510263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_300['answer'] = responses
/tmp/ipython-input-38-2805510263.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_300['doc_citations'] = df_300['answer'].apply(extract_doc_citations)


,question,answer,docs,doc_citations
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도 중남부를 약 한 ...,[“물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극...,[doc4]
1,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는 과학기술정보통신부입니다...,[청강문화산업대는 지난해 5월 컴퓨터그래픽(CG) 소프트웨어(SW)업체 에프엑스기어...,[doc4]
2,로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,로버트 헨리 딕은 1946년에 매사추세츠 공과대학교 방사선 연구소에서 레이다를 개발...,[사이코-사이버네틱스(Psycho-Cybernetics)는 정신적인 자동유도장치라는...,[doc2]
3,개막전에서 3안타 2실점을 기록해서 패한 선수는?,개막전에서 3안타 2실점을 기록해서 패한 선수는 사이타마 세이부 라이온스의 와쿠이 ...,[시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인...,[doc1]
4,컵라면 매출에서 불닭볶음면을 이긴 상품은?,컵라면 매출에서 불닭볶음면을 이긴 상품은 세븐일레븐에서 작년 10월에 출시된 ‘교동...,[영화관 안에서만 주로 접했던 팝콘이 편의점 진열대의 윗자리를 차지하는 대표 스낵으...,[doc5]


In [ ]:
# 인용 문서 정보가 없는 행 제거 (즉, doc_citations가 빈 리스트인 행 제거)
df_with_citation = df_300[df_300['doc_citations'].apply(lambda x: len(x) > 0)].reset_index(drop=True)

print(f"인용 정보가 있는 행만 남은 데이터 shape: {df_with_citation.shape}")

인용 정보가 있는 행만 남은 데이터 shape: (294, 8)


In [ ]:
df_with_citation[['question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']]

,question,docs,user_prompt,system_prompt,answer,doc_citations
0,북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?,[“물빛만 봐도 동해는 한눈에 알아볼 수 있습니다.” 한국 국적선 사상 최초로 북극...,질문: 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?\n\ndoc...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",북태평양 기단과 오호츠크해 기단이 만나 형성되는 장마전선은 한반도 중남부를 약 한 ...,[doc4]
1,성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?,[청강문화산업대는 지난해 5월 컴퓨터그래픽(CG) 소프트웨어(SW)업체 에프엑스기어...,질문: 성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는?\n\ndocs...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",성공적인 성과를 보인 지역SW서비스사업화 지원사업의 주최자는 과학기술정보통신부입니다...,[doc4]
2,로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?,[사이코-사이버네틱스(Psycho-Cybernetics)는 정신적인 자동유도장치라는...,질문: 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?\...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",로버트 헨리 딕은 1946년에 매사추세츠 공과대학교 방사선 연구소에서 레이다를 개발...,[doc2]
3,개막전에서 3안타 2실점을 기록해서 패한 선수는?,[시범 경기에서는 16이닝을 던져 15실점을 기록하는 등 성적이 좋지 않았지만 본인...,질문: 개막전에서 3안타 2실점을 기록해서 패한 선수는?\n\ndocs:\ndoc1...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",개막전에서 3안타 2실점을 기록해서 패한 선수는 사이타마 세이부 라이온스의 와쿠이 ...,[doc1]
4,컵라면 매출에서 불닭볶음면을 이긴 상품은?,[영화관 안에서만 주로 접했던 팝콘이 편의점 진열대의 윗자리를 차지하는 대표 스낵으...,질문: 컵라면 매출에서 불닭볶음면을 이긴 상품은?\n\ndocs:\ndoc1: 영화...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",컵라면 매출에서 불닭볶음면을 이긴 상품은 세븐일레븐에서 작년 10월에 출시된 ‘교동...,[doc5]
...,...,...,...,...,...,...
289,애류헌에서 판매되는 전통주는 몇 종인가?,[국립종자원은 이달 초 제주도에 지원을 개설했다. 아열대 작물을 개량해 특허권을 확...,질문: 애류헌에서 판매되는 전통주는 몇 종인가?\n\ndocs:\ndoc1: 국립종...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",애류헌에서 판매되는 전통주의 종류에 대해서는 주어진 문서들 중에 구체적인 숫자가 명...,[doc4]
290,마쓰카타 마사요시의 재정정책으로 가장 많은 타격을 입은 산업분야는?,[막말 이후 부국강병 명분으로 해마다 증세가 이뤄지는 가운데 1881년 재무경에 취...,질문: 마쓰카타 마사요시의 재정정책으로 가장 많은 타격을 입은 산업분야는?\n\nd...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...","마쓰카타 마사요시의 재정정책으로 가장 많은 타격을 입은 산업분야는 농업, 특히 양잠...",[doc1]
291,다중 계정 사용자의 판별을 불가능하게 하는 것은?,[개인정보 불법 거래가 여전한 데다 금융사기가 증가세를 보이면서 ‘금융사기의 필수품...,질문: 다중 계정 사용자의 판별을 불가능하게 하는 것은?\n\ndocs:\ndoc1...,"당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",다중 계정 사용자의 판별을 불가능하게 하는 것은 공개 프록시를 사용하는 경우입니다....,[doc4]
292,가시리잇고가 상영을 시작한 달은?,"[‘일본의, 일본에 의한, 일본을 위한 애니메이션.’일본 미야자키 하야오 감독(72...","질문: 가시리잇고가 상영을 시작한 달은?\n\ndocs:\ndoc1: ‘일본의, 일...","당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, ...",가시리잇고는 2021년 1월에 방영을 시작할 예정입니다. 구체적으로 2021년 1월...,[doc2]


In [ ]:
# 저장할 컬럼만 추출
save_cols = ['question', 'docs', 'user_prompt', 'system_prompt', 'answer', 'doc_citations']

# 파일로 저장 (utf-8, 인덱스 없이)
df_with_citation.to_csv(
    'klue_mrc_prompt_docs_5_answer_citations.csv',
    columns=save_cols,
    index=False,
    encoding='utf-8'
)


In [ ]:
from google.colab import files
files.download('klue_mrc_prompt_docs_5_answer_citations.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>